In [1]:
from keybert import KeyBERT
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import pandas as pd
import torch
from tqdm import tqdm, trange
from random import randint
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
tqdm.pandas()

In [3]:
def morphs(text, noun = True, verb = False, adjective = False, adverb = False):
    tokens = word_tokenize(text)
    poses = pos_tag(tokens, tagset = 'universal')
    filters = []

    if noun:
        filters.append('NOUN')
    if verb:
        filters.append('VERB')
    if adjective:
        filters.append('ADJ')
    if adverb:
        filters.append('ADV')

    return [pos[0] for pos in poses if pos[1] in filters]

In [4]:
model = KeyBERT('distilbert-base-nli-mean-tokens')

In [5]:
df1 = pd.read_csv('./datasets/roblox1.csv', index_col = 0)
df2 = pd.read_csv('./datasets/roblox2.csv', index_col = 0)
df3 = pd.read_csv('./datasets/roblox3.csv', index_col = 0)
df4 = pd.read_csv('./datasets/roblox4.csv', index_col = 0)
df5 = pd.read_csv('./datasets/roblox5.csv', index_col = 0)
df6 = pd.read_csv('./datasets/zepeto.csv', index_col = 0)
df = pd.concat([df1, df2, df3, df4, df5, df6]).reset_index(drop = True)

/tmp/ipykernel_2565152/2336632862.py:2: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('./datasets/roblox2.csv', index_col = 0)
/tmp/ipykernel_2565152/2336632862.py:3: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('./datasets/roblox3.csv', index_col = 0)
/tmp/ipykernel_2565152/2336632862.py:4: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv('./datasets/roblox4.csv', index_col = 0)


In [6]:
df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,05426917-05d0-4cce-89d7-27fa81c313f3,LT. Silver,https://play-lh.googleusercontent.com/a-/AD_cM...,As a person who has played this game for many ...,4,749,2.581.563,2023-07-03 04:51:21,NaN,NaN,2.581.563
1,1ef0a865-6bb8-4d2b-911c-ac0603ec3091,Hiba Nur,https://play-lh.googleusercontent.com/a/AAcHTt...,"Overall the game is pretty good, but there are...",4,5612,2.581.563,2023-06-27 11:54:39,NaN,NaN,2.581.563
2,6822f649-eeb4-420b-9630-920709c741c7,Graham Purdue,https://play-lh.googleusercontent.com/a/AAcHTt...,"It's a little mixed for me, the interface is k...",4,347,2.581.563,2023-07-06 07:27:37,NaN,NaN,2.581.563
3,350fe8a7-7acd-4520-8382-1e8b3fa60b06,Amber Wood,https://play-lh.googleusercontent.com/a-/AD_cM...,I have been playing this game for 6 years sinc...,3,1202,2.581.563,2023-07-05 06:06:27,NaN,NaN,2.581.563
4,1c0c638e-8c94-4421-bd3c-0b13d2f1aed5,Sophia Lewis,https://play-lh.googleusercontent.com/a/AAcHTt...,"Hello, I like on how it is very limitless and ...",3,379,2.581.563,2023-06-28 12:30:25,NaN,NaN,2.581.563
...,...,...,...,...,...,...,...,...,...,...,...
562102,216320bc-49e0-45f4-9e1f-c006e9f8d17a,Joana angela Lopez,https://play-lh.googleusercontent.com/a/AAcHTt...,Yumica Yumica Yumica Yumica,5,0,NaN,2021-10-13 01:47:32,NaN,NaN,NaN
562103,7f402bb6-22dd-4306-9448-36bfc7835f53,EliD,https://play-lh.googleusercontent.com/a-/AD_cM...,ZEPETO ZEPETO ZEPETO ZEPETO ZEPETO,5,0,NaN,2021-09-09 05:04:37,"Have a lovely time! EliD, Thanks for playing Z...",2021-09-09 10:28:51,NaN
562104,fa33d0ac-54a4-4886-9833-ecbee1bc1211,B Das,https://play-lh.googleusercontent.com/a-/AD_cM...,Hehe Hehe Hehe Hehe Hehe,5,0,NaN,2021-06-14 15:45:39,NaN,NaN,NaN
562105,78c46bb9-741e-41ec-8d1e-b1efe69d3059,vv hgg,https://play-lh.googleusercontent.com/a/AAcHTt...,កកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកកក...,5,0,NaN,2021-11-26 22:22:01,NaN,NaN,NaN


In [7]:
df['keybert_keywords'] = df['content'].progress_apply(lambda x : model.extract_keywords(x, top_n = 10))

  6%|▋         | 35322/562107 [39:55<9:55:30, 14.74it/s] 


KeyboardInterrupt: 